### 1. Import packages

First, let's import needed modules and, random seed (we'll use it if needed) and create some auxiliary functions.

In [1]:
import pandas as pd
import numpy as np
from itertools import islice, chain
from nltk import ngrams
from tqdm.notebook import tqdm
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Tuple, Iterator

SEED=42

In [2]:
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

### 2. Data preparation

I'll be using dataset from [Spooky Author Identification](https://www.kaggle.com/c/spooky-author-identification/overview) competition

#### 2.1 Loading the data

In [3]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

#### 2.2 Data Fields

* id - a unique identifier for each sentence
* text - some text written by one of the authors
* author - the author of the sentence (EAP: Edgar Allan Poe, HPL: HP Lovecraft; MWS: Mary Wollstonecraft Shelley)

Let's look at the data

In [4]:
train_df.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


For now, I'm going to look only at column text to look at the ways text representation can be done

#### 2.3 Data Splitting

But nevertheless let's split the data into training and validation sets.  
As soon as we have almost $20 000$ rows in `train_df`, test size will be limited to $10\%$

In [5]:
train, val = train_test_split(train_df, test_size=0.1, random_state=SEED)

### 3. Word embeddings

#### 3.1 Bag of words

##### 3.1.1 One-hot vectors 

The simplest way of word representation is **one-hot vectors**. For the i-th word in the vocabulary, the vector has 1 on the i-th dimension and 0 on the rest.   
Let's do this using sklearn's `CountVectorizer` with `binary=True`

In [6]:
count_vect = CountVectorizer(binary=True)
X_train_oh = count_vect.fit_transform(train['text'])
print(f"The size of the train dataset is {X_train_oh.shape}")

The size of the train dataset is (17621, 24069)


By default, we are not limiting the vocabulary of the model and the length of the vector for every sentence will be $24$ $069$ - number of words in our vocab.
Although, it can be done by setting parameter `max_features` to, for example, $10000$. By doing this, vocabulary will be built considering only the top `max_features` ordered by term frequency across the corpus.

In [7]:
X_train_oh

<17621x24069 sparse matrix of type '<class 'numpy.int64'>'
	with 386417 stored elements in Compressed Sparse Row format>

It is also worth to mention, that due to the sparsity of representation (most values in word vectors will be zeros) we can save a lot of memory by only storing the non-zero parts of the feature vectors in memory. `scipy.sparse` matrices are data structures that do exactly this and they are used in `sklearn` package.

In [8]:
17621*24069

424119849

In our case only $386$ $417$ of elements out of $424$ $119$ $849$ are non-zero.

In [9]:
count_vect_lim_vocab = CountVectorizer(binary=True, max_features=10_000)
X_train_oh = count_vect_lim_vocab.fit_transform(train['text'])
X_train_oh.shape

(17621, 10000)

Now, the length of the vector is $10000$.

Let's look at the vector for the first text in our train corpus.

In [10]:
first_sentence = val['text'][6148]

In [11]:
one_hot_vector = count_vect.transform([first_sentence])
one_hot_vector

<1x24069 sparse matrix of type '<class 'numpy.int64'>'
	with 16 stored elements in Compressed Sparse Row format>

In [12]:
one_hot_vector = one_hot_vector.toarray()
one_hot_vector

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

It is a sparse vector with $24$ $069$ elements with only $16$ elements that are not equal to zero. 
Let's find out which are they.

In [13]:
indices = np.where(np.any(one_hot_vector!=0, axis=0))[0]
indices

array([  614,   813,  1548,  2128,  5189,  9205, 10643, 12834, 14023,
       14557, 20543, 21197, 21300, 21556, 22713, 23613], dtype=int64)

These are the indices of words which are present in our sentence.  
Now we are going to create index-to-word dictionary to check the result of the work of `CountVectorizer`

In [14]:
index_to_word = {index : word for word, index in count_vect.vocabulary_.items()}
dict(take(10, index_to_word.items()) )

{20160: 'still',
 14787: 'others',
 10869: 'including',
 11721: 'joe',
 10157: 'himself',
 9894: 'have',
 21222: 'theories',
 21548: 'too',
 23652: 'wild',
 813: 'and'}

In [15]:
[(ind, index_to_word[ind], one_hot_vector[0, ind]) for ind in indices]

[(614, 'all', 1),
 (813, 'and', 1),
 (1548, 'available', 1),
 (2128, 'bewildered', 1),
 (5189, 'dazzled', 1),
 (9205, 'gigantic', 1),
 (10643, 'immediately', 1),
 (12834, 'magnitude', 1),
 (14023, 'nature', 1),
 (14557, 'of', 1),
 (20543, 'sum', 1),
 (21197, 'the', 1),
 (21300, 'thought', 1),
 (21556, 'topic', 1),
 (22713, 'upon', 1),
 (23613, 'who', 1)]

Indeed, these are the indices and corresponding words from our sentence

Because we've created the `CountVectorizer` with `binary=True`. The elements are really ones and zeros.  

##### 3.1.2 One-hot vectors with counts

A little improvement over that will be using vectorizer with `binary=False`, because this way we will take counts into account.

In [16]:
count_vect = CountVectorizer(binary=False)
X_train_cv = count_vect.fit_transform(train['text'])

one_hot_vector = count_vect.transform([first_sentence]).toarray()
indices = np.where(np.any(one_hot_vector!=0, axis=0))[0]
index_to_word = {index : word for word, index in count_vect.vocabulary_.items()}

[(ind, index_to_word[ind], one_hot_vector[0, ind]) for ind in indices]

[(614, 'all', 1),
 (813, 'and', 2),
 (1548, 'available', 1),
 (2128, 'bewildered', 1),
 (5189, 'dazzled', 1),
 (9205, 'gigantic', 1),
 (10643, 'immediately', 1),
 (12834, 'magnitude', 1),
 (14023, 'nature', 1),
 (14557, 'of', 1),
 (20543, 'sum', 1),
 (21197, 'the', 4),
 (21300, 'thought', 1),
 (21556, 'topic', 1),
 (22713, 'upon', 1),
 (23613, 'who', 1)]

We can see that now the value for 'the' is 4 and for 'two' it is 2. It doesn't only show that these words are present in the sentence, but also indicate how many times they occur in the sentence.

In [17]:
tokenizer = count_vect.build_tokenizer()
tokenized_sentence = tokenizer(first_sentence.lower())
tokenized_sentence.count('the')

4

##### 3.1.3 N-grams

We can take into account not only words, but collocations using parameter `ngram_range` to preserve some local ordering, because by using unigrams we don't capture even that.  

In [18]:
bigram_count_vect = CountVectorizer(binary=False, ngram_range=(1,2))
X_train_bigram = bigram_count_vect.fit_transform(train['text'])
print(f"The size of the train dataset is {X_train_bigram.shape}")
print(X_train_bigram.count_nonzero)
one_hot_vector = bigram_count_vect.transform([first_sentence]).toarray()
indices = np.where(np.any(one_hot_vector!=0, axis=0))[0]
index_to_word = {index : word for word, index in bigram_count_vect.vocabulary_.items()}

print([(ind, index_to_word[ind], one_hot_vector[0, ind]) for ind in indices])

The size of the train dataset is (17621, 230440)
<bound method _data_matrix.count_nonzero of <17621x230440 sparse matrix of type '<class 'numpy.int64'>'
	with 813907 stored elements in Compressed Sparse Row format>>
[(4324, 'all', 1), (4855, 'all who', 1), (7667, 'and', 2), (8116, 'and bewildered', 1), (12202, 'and the', 1), (19515, 'available', 1), (25467, 'bewildered', 1), (45745, 'dazzled', 1), (75382, 'gigantic', 1), (93018, 'immediately', 1), (112540, 'magnitude', 1), (112541, 'magnitude and', 1), (124876, 'nature', 1), (124942, 'nature of', 1), (131308, 'of', 1), (134836, 'of the', 1), (181040, 'sum', 1), (186670, 'the', 4), (189413, 'the gigantic', 1), (192991, 'the sum', 1), (193291, 'the topic', 1), (199054, 'thought', 1), (199173, 'thought upon', 1), (203935, 'topic', 1), (209892, 'upon', 1), (210124, 'upon the', 1), (222194, 'who', 1), (222471, 'who thought', 1)]


By including bigrams into calculations the size of the vector increased from $24$ $069$ to $230$ $440$, but we still can limit it using `max_features` parameter, so it is not a big deal.

##### 3.1.4 Stopwords

Removing or not removing stopwords is a controversial topic...

##### 3.1.5 Advantages and disadvantages

$"+"$:
1. Fast to train (actually, there is no training done - just counting)

$"-"$:
1. Vector dimensionality is equal to vocabulary size (`max_features`) 
2. Longer documents will have higher average count values than shorter documents, even though they might talk about the same topics
3. **One-hot vectors don't capture meaning**

##### 3.1.6 Baseline model

To evaluate embeddings we will stick to extrinsic evaluation - evaluation on real task - text classification

In [19]:
count_vect_eval = CountVectorizer(ngram_range=(1,1))
X_train = count_vect_eval.fit_transform(train['text'])
X_val = count_vect_eval.transform(val['text'])

In [20]:
logreg_count = LogisticRegression(solver='sag', max_iter=10000)
logreg_count.fit(X_train, train['author'])

LogisticRegression(max_iter=10000, solver='sag')

In [21]:
pred = logreg_count.predict(X_val)
f1 = f1_score(pred, val['author'], average='macro')
f1

0.8226872991519389

#### 3.2 TF-IDF

##### 3.2.1 Understanding

To address second issue of bag of words approach and in attempt of trying to deal with common words (stopwords) without deleting them the next approach can be used.  

Tf means term-frequency while tf–idf means term-frequency times inverse document-frequency:
$$ \large tf-idf(t,d) = tf(i,d) * idf(t)$$  
Term frequency is: 
$$ \large tf(i,d) = \frac{wordCount(t,d)}{length(d)}$$ 
where:
* `wordCount(t,d)` is the number of occurrences of term *t* in the document *d*
* `length(d)` is the number of words in the document *d*  

Dividing the number of occurrences of each word in a document by the total number of words in the document helps to solve the issue mentioned in the first sentence of this block.

Inverse document-frequency is:
$$ \large idf(t, c) = \frac{size(c)}{docCount(t, c)}$$ 
where: 
* `size(c)` is the number of documents in the corpora *c*
* `docCount(t, c)` is the number of documents in corpora *c* containing the term *t*  

Ifd helps to downscale weights for words that occur in many documents in the corpus and are therefore less informative than those that occur only in a smaller portion of the corpus.

##### 3.2.2 Real life form

In real life tf-idf is computed differently:
$$ \large tf(i,d) = \log{ \left(1 + \frac{wordCount(t,d)}{length(d)}\right)}$$ 
with `sublinear_tf=True` for `sklearn.feature_extraction.text.TfidfVectorizer`
$$ \large idf(t, c) = 1 + \log{\left(\frac{1 + size(c)}{1 + docCount(t, c)}\right)}$$ 
with `norm='l2', use_idf=True, smooth_idf=True` for `sklearn.feature_extraction.text.TfidfVectorizer`

The reason to use `log` here is this. 

After that the vector with tf-idf scores for the words in the sentence should be normalized by the Euclidean norm:
$$\large v_{norm} = \frac{v}{||v||_2} = \frac{v}{\sqrt{v_{1}^2 + v_{2}^2 + ... + v_{n}^2}}$$
By doing this each output vector will have l2 unit norm. The cosine similarity between two vectors is their dot product when l2 norm has been applied.

##### 3.2.3 Sklearn's implementation

In [22]:
tfidf_vect = TfidfVectorizer()
X_train_tf_idf = tfidf_vect.fit_transform(train['text'])
print(f"The size of the train dataset is {X_train_tf_idf.shape}")

The size of the train dataset is (17621, 24069)


In [23]:
X_train_tf_idf

<17621x24069 sparse matrix of type '<class 'numpy.float64'>'
	with 386417 stored elements in Compressed Sparse Row format>

In [24]:
tfidf_vect.transform([first_sentence])

<1x24069 sparse matrix of type '<class 'numpy.float64'>'
	with 16 stored elements in Compressed Sparse Row format>

In [25]:
index_to_word = {index : word for word, index in tfidf_vect.vocabulary_.items()}

In [26]:
tfidf_vector = tfidf_vect.transform([first_sentence])
tfidf_vector

<1x24069 sparse matrix of type '<class 'numpy.float64'>'
	with 16 stored elements in Compressed Sparse Row format>

In [27]:
tfidf_vector = tfidf_vector.toarray()
tfidf_vector

array([[0., 0., 0., ..., 0., 0., 0.]])

In [28]:
indices = np.where(np.any(tfidf_vector!=0, axis=0))[0]
indices

array([  614,   813,  1548,  2128,  5189,  9205, 10643, 12834, 14023,
       14557, 20543, 21197, 21300, 21556, 22713, 23613], dtype=int64)

In [29]:
[(ind, index_to_word[ind], tfidf_vector[0, ind]) for ind in indices]

[(614, 'all', 0.13031453897504672),
 (813, 'and', 0.12214845066990851),
 (1548, 'available', 0.3924242175008267),
 (2128, 'bewildered', 0.3114996887796031),
 (5189, 'dazzled', 0.36544937459375215),
 (9205, 'gigantic', 0.2695648987205094),
 (10643, 'immediately', 0.23647294743920957),
 (12834, 'magnitude', 0.32608143594739347),
 (14023, 'nature', 0.2049931029681656),
 (14557, 'of', 0.059811003957834306),
 (20543, 'sum', 0.3009169879304377),
 (21197, 'the', 0.20393197647953104),
 (21300, 'thought', 0.18711801923161203),
 (21556, 'topic', 0.2973766872628445),
 (22713, 'upon', 0.14506693111726507),
 (23613, 'who', 0.16239686486865726)]

We see that the words that are popular among the sentences in the corpus have a much lower tf-idf score, than less popular ones.
For example, words that have:
* lowest scores: all, an, of, upon *occur often in the sentences, thus cannot help to discern the sentences between themselves*.
* highest scores: available, dazzled, magnitude, sum, topic *are probably rarely found in the sentences in the corpus of texts, but are available in this sentence and thus will make a good features*.

##### 3.2.4 TF-IDF implementation

I'm going to implement tf-idf from scratch and compare results with sklearn's implementation.  
However, to save time I will use already trained tokenizer from sklearn.  
*Disclaimer: this implementation is not optimized and not tested properly*

In [30]:
tfidf_tokenizer = tfidf_vect.build_tokenizer()
first_sentence_tokenized = tfidf_tokenizer(first_sentence.lower())
first_sentence_tokenized_set = list(set(first_sentence_tokenized))

I will get the scores for the same sentence 'The gigantic magnitude and the immediately available nature of the sum, dazzled and bewildered all who thought upon the topic.'

In [31]:
def get_tf(tokenized_text, term):
    return tokenized_text.count(term) / len(tokenized_text)

def get_idf(tokenized_corpus, term, tokenizer):
    size = len(tokenized_corpus)
    doc_count = sum([1 if term in text else 0 for text in tokenized_corpus])
    
    return np.log((1 + size) / (1 + doc_count)) + 1

def get_tf_idf(tf, idf):
    return tf * idf

def get_l2_norm(vector):
    return np.sqrt(np.sum(vector**2))

In [32]:
tf_idf_first_sentece = []
tokenized_corpus = [tokenizer(text.lower()) for text in train['text']]

for word in first_sentence_tokenized_set:
    tf_ = get_tf(first_sentence_tokenized, word)
    idf_ = get_idf(tokenized_corpus, word, tfidf_tokenizer)
    tf_idf_first_sentece.append(get_tf_idf(tf_, idf_))
    
tf_idf_first_sentece = np.array(tf_idf_first_sentece)
l2_norm = get_l2_norm(tf_idf_first_sentece)
tf_idf_first_sentece = tf_idf_first_sentece / l2_norm

In [33]:
tf_idf_first_sentence_dict = dict(zip(first_sentence_tokenized_set, tf_idf_first_sentece))

In [34]:
[(ind, index_to_word[ind], tf_idf_first_sentence_dict[index_to_word[ind]]) for ind in indices]

[(614, 'all', 0.13031453897504672),
 (813, 'and', 0.12214845066990852),
 (1548, 'available', 0.3924242175008267),
 (2128, 'bewildered', 0.3114996887796031),
 (5189, 'dazzled', 0.36544937459375215),
 (9205, 'gigantic', 0.2695648987205094),
 (10643, 'immediately', 0.23647294743920957),
 (12834, 'magnitude', 0.32608143594739347),
 (14023, 'nature', 0.2049931029681656),
 (14557, 'of', 0.05981100395783431),
 (20543, 'sum', 0.3009169879304377),
 (21197, 'the', 0.20393197647953104),
 (21300, 'thought', 0.18711801923161206),
 (21556, 'topic', 0.2973766872628445),
 (22713, 'upon', 0.1450669311172651),
 (23613, 'who', 0.16239686486865726)]

In the end, we are getting the same results as in sklearn's implementation with default parameters.

##### 3.2.5 Main parameters

* `strip_accents`: Remove accents and perform other character normalization during the preprocessing step. 
* `stop_words`: If a string, it is passed to _check_stop_list and the appropriate stop list is returned. ‘english’ is currently the only supported string value.
* `ngram_range`: The lower and upper boundary of the range of n-values for different n-grams to be extracted. All values of n such that min_n <= n <= max_n will be used. For example an ngram_range of (1, 1) means only unigrams, (1, 2) means unigrams and bigrams, and (2, 2) means only bigrams.
* `max_df`: When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float in range [0.0, 1.0], the parameter represents a proportion of documents, integer absolute counts.
* `min_df`: When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float in range of [0.0, 1.0], the parameter represents a proportion of documents, integer absolute counts.
* `max_features`: If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.
* `binary`: If True, all non-zero term counts are set to 1. This does not mean outputs will have only 0/1 values, only that the tf term in tf-idf is binary. (Set idf and normalization to False to get 0/1 outputs).
* `norm`: Each output row will have unit norm, either:
    * ‘l2’: Sum of squares of vector elements is 1. The cosine similarity between two vectors is their dot product when l2 norm has been applied.
    * ‘l1’: Sum of absolute values of vector elements is 1. See preprocessing.normalize.
* `use_idf`: Enable inverse-document-frequency reweighting. If False, idf(t) = 1.
* `smooth_idf`: Smooth idf weights by adding one to document frequencies, as if an extra document was seen containing every term in the collection exactly once. Prevents zero divisions.
* `sublinear_tf`: Apply sublinear tf scaling, i.e. replace tf with 1 + log(tf).



Hyperparameters to use for optimization:
* max_df
* min_df
* max_features
* ngram_range

##### 3.2.6 N-grams

In [35]:
bigram_tfidf_vect = TfidfVectorizer(ngram_range=(1,2))

X_train_bigram_tfidf = bigram_tfidf_vect.fit_transform(train['text'])

bigram_tfidf_vector = bigram_tfidf_vect.transform([first_sentence]).toarray()
indices = np.where(np.any(bigram_tfidf_vector!=0, axis=0))[0]
index_to_word = {index : word for word, index in bigram_tfidf_vect.vocabulary_.items()}

print(f"The size of the train dataset is {X_train_bigram_tfidf.shape}")
print(X_train_bigram_tfidf.count_nonzero)
print(first_sentence)

[(ind, index_to_word[ind], bigram_tfidf_vector[0, ind]) for ind in indices]

The size of the train dataset is (17621, 230440)
<bound method _data_matrix.count_nonzero of <17621x230440 sparse matrix of type '<class 'numpy.float64'>'
	with 813907 stored elements in Compressed Sparse Row format>>
The gigantic magnitude and the immediately available nature of the sum, dazzled and bewildered all who thought upon the topic.


[(4324, 'all', 0.088547269675905),
 (4855, 'all who', 0.2335206648733459),
 (7667, 'and', 0.0829985041349335),
 (8116, 'and bewildered', 0.24831875690585964),
 (12202, 'and the', 0.09948207535251821),
 (19515, 'available', 0.26664786053577216),
 (25467, 'bewildered', 0.21166054964603465),
 (45745, 'dazzled', 0.24831875690585964),
 (75382, 'gigantic', 0.18316632948172856),
 (93018, 'immediately', 0.16068071922477745),
 (112540, 'magnitude', 0.22156868352708564),
 (112541, 'magnitude and', 0.24831875690585964),
 (124876, 'nature', 0.13929051748936838),
 (124942, 'nature of', 0.1894923535718752),
 (131308, 'of', 0.040640907290130535),
 (134836, 'of the', 0.06654221632472784),
 (181040, 'sum', 0.20446972294810256),
 (186670, 'the', 0.13856949392524126),
 (189413, 'the gigantic', 0.24831875690585964),
 (192991, 'the sum', 0.2335206648733459),
 (193291, 'the topic', 0.23759691861052884),
 (199054, 'thought', 0.12714459829609193),
 (199173, 'thought upon', 0.2559260222404413),
 (203935, 'topi

Using bigrams (trigrams) will probably greatly improve the performance of the solution based on tf-idf embeddings.  

##### 3.2.7 Advantages and disadvantages

$"+"$:
1. Outperforms 'bag of words' approach and solves some of its problems

$"-"$:
1. Still doesn't capture meaning

Generally, Tf-Idf can be used as a baseline technique for producing text embeddings and text classification (for example, tf-idf + LogReg with hyperparameter optimization).


##### 3.2.8 Baseline model

To evaluate embeddings we will stick to extrinsic evaluation - evaluation on real task - text classification

In [36]:
tfidf_vect_eval = TfidfVectorizer(ngram_range=(1,1))
X_train_tfidf = tfidf_vect_eval.fit_transform(train['text'])
X_val_tfidf = tfidf_vect_eval.transform(val['text'])

In [37]:
logreg_tfidf = LogisticRegression(solver='sag', max_iter=10000)
logreg_tfidf.fit(X_train_tfidf, train['author'])

LogisticRegression(max_iter=10000, solver='sag')

In [38]:
pred = logreg_tfidf.predict(X_val_tfidf)
f1 = f1_score(pred, val['author'], average='macro')
f1

0.816697787096243

#### 3.3 Distributional Semantics

Words which frequently appear in similar contexts have similar meaning.

Often you can find it formulated as "You shall know a word by the company it keeps" with the reference to J. R. Firth in 1957, but actually there were a lot more people responsible, and much earlier. For example, Harris, 1954.

This is an extremely valuable idea: it can be used in practice to make word vectors capture their meaning. According to the distributional hypothesis, "to capture meaning" and "to capture contexts" are inherently the same. Therefore, all we need to do is to put information about word contexts into word representation.

Main idea: We need to put information about word contexts into word representation.

[Distributional Semantics from Lena Voita's NLP Course | For You](https://lena-voita.github.io/nlp_course/word_embeddings.html#distributional_semantics)

#### 3.4 Count-Based Methods

Text about count-based methods

##### 3.4.1 Co-Occurence Counts

Text about Co-Occurence Counts

In [94]:
tokenized_train = [tokenizer(text.lower()) for text in train['text']]
tokenized_val = [tokenizer(text.lower()) for text in val['text']]

In [95]:
unique_words = sorted(list(set(chain.from_iterable(tokenized_train))))

In [96]:
cooccurrence_dataframe = pd.DataFrame(0, columns=unique_words, index=unique_words)

In [97]:
tokenized_train[0]

['still',
 'others',
 'including',
 'joe',
 'himself',
 'have',
 'theories',
 'too',
 'wild',
 'and',
 'fantastic',
 'for',
 'sober',
 'credence']

In [98]:
cooccurrence_dataframe.loc['still', 'others']

0

In [99]:
def get_window(text: List[str], window_size: int) -> Iterator[Tuple[str, List[str]]]:
    for backward, current in enumerate(range(len(text)), start=0 - (window_size // 2)):
        if backward < 0:
            backward = 0
        context = list(text[backward:current]) + list(text[current + 1:current + 1 + window_size // 2])
        center = text[current]
        yield center, context

In [100]:
for center, context_values in get_window(tokenized_train[0], 5):
    print(center, context_values)

still ['others', 'including']
others ['still', 'including', 'joe']
including ['still', 'others', 'joe', 'himself']
joe ['others', 'including', 'himself', 'have']
himself ['including', 'joe', 'have', 'theories']
have ['joe', 'himself', 'theories', 'too']
theories ['himself', 'have', 'too', 'wild']
too ['have', 'theories', 'wild', 'and']
wild ['theories', 'too', 'and', 'fantastic']
and ['too', 'wild', 'fantastic', 'for']
fantastic ['wild', 'and', 'for', 'sober']
for ['and', 'fantastic', 'sober', 'credence']
sober ['fantastic', 'for', 'credence']
credence ['for', 'sober']


step 1: construct a word-context matrix

In [101]:
window_size = 5
for tokenized_sentence in tqdm(tokenized_train):
    for center, context_values in get_window(tokenized_sentence, window_size):
        for context_word in context_values:
            cooccurrence_dataframe.at[center, context_word] += 1

  0%|          | 0/17621 [00:00<?, ?it/s]

In [102]:
cooccurrence_dataframe.loc['still', 'others']

2

step 2: reduce its dimensionality

In [48]:
X = csr_matrix(cooccurrence_dataframe)
svd = TruncatedSVD(n_components=200, n_iter=7, random_state=SEED)
svd.fit(X)

TruncatedSVD(n_components=200, n_iter=7, random_state=42)

In [49]:
word_vectors = svd.transform(X)
word_vectors.shape

(24069, 200)

step 3: normalize word vectors

In [50]:
from numpy.linalg import norm

In [51]:
norm(word_vectors, axis=1)

array([1.00427691, 2.62921217, 1.41296201, ..., 2.23500891, 2.44693   ,
       0.99934727])

In [52]:
word_vectors /= norm(word_vectors, axis=1).reshape(-1,1)

In [53]:
norm(word_vectors, axis=1)

array([1., 1., 1., ..., 1., 1., 1.])

##### 3.4.1.1 Baseline model

To evaluate embeddings we will stick to extrinsic evaluation - evaluation on real task - text classification

In [54]:
embeddings_lookup = dict(zip(cooccurrence_dataframe.index.values, word_vectors))

In [55]:
def sent2vec(tok_sentence, embeddings_lookup):
    word_embeddings = []
    for word in tok_sentence:
        try:
            word_embeddings.append(embeddings_lookup[word])
        except:
            continue
    word_embeddings = np.array(word_embeddings)
    v = word_embeddings.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(100)
    return v / np.sqrt((v ** 2).sum())

In [56]:
X_train_cooc_counts = [sent2vec(tokenized_sentence, embeddings_lookup) for tokenized_sentence in tqdm(tokenized_train)]
X_val_cooc_counts = [sent2vec(tokenized_sentence, embeddings_lookup) for tokenized_sentence in tqdm(tokenized_val)]

  0%|          | 0/17621 [00:00<?, ?it/s]

  0%|          | 0/1958 [00:00<?, ?it/s]

In [57]:
logreg_cooc_counts = LogisticRegression(solver='sag', max_iter=10000)
logreg_cooc_counts.fit(X_train_cooc_counts, train['author'])

LogisticRegression(max_iter=10000, solver='sag')

In [58]:
pred = logreg_cooc_counts.predict(X_val_cooc_counts)
f1 = f1_score(pred, val['author'], average='macro')
f1

0.6513907093968357

intrinsic evaluation

In [59]:
embedding_1 = embeddings_lookup['remain'].reshape(1, -1)
embedding_2 = embeddings_lookup['stay'].reshape(1, -1)

In [60]:
sim = cosine_similarity(embedding_1, embedding_2)
sim

array([[0.82007657]])

##### 3.4.2 Positive Pointwise Mutual Information (PPMI)

Theory

In [89]:
pmi_dataframe = cooccurrence_dataframe

In [111]:
sum_cooccurrences = cooccurrence_dataframe.to_numpy().sum()
row_counts = cooccurrence_dataframe.sum(axis=0).to_dict()
column_counts = cooccurrence_dataframe.sum(axis=1).to_dict()

In [116]:
window_size = 5
for tokenized_sentence in tqdm(tokenized_train):
    for center, context_values in get_window(tokenized_sentence, window_size):
        for context_word in context_values:
            p_i = row_counts[center] / sum_cooccurrences
            p_j = column_counts[context_word] / sum_cooccurrences
            p_ij = cooccurrence_dataframe.at[center, context_word] / sum_cooccurrences
            pmi_dataframe.at[center, context_word] = np.log2(p_ij / (p_i * p_j))

  0%|          | 0/17621 [00:00<?, ?it/s]

In [128]:
ppmi_dataframe = pmi_dataframe

In [129]:
ppmi_dataframe[:] = np.vectorize(lambda value: max(0, value))(ppmi_dataframe)

In [130]:
ppmi_dataframe.loc['still', 'others']

2

step 2: reduce its dimensionality

In [131]:
X = csr_matrix(ppmi_dataframe)
svd = TruncatedSVD(n_components=200, n_iter=7, random_state=SEED)
svd.fit(X)

TruncatedSVD(n_components=200, n_iter=7, random_state=42)

In [132]:
word_vectors = svd.transform(X)
word_vectors.shape

(24069, 200)

step 3: normalize word vectors

In [133]:
from numpy.linalg import norm

In [134]:
norm(word_vectors, axis=1)

array([1.51575276, 7.66269329, 4.17084946, ..., 3.33118166, 4.66171782,
       2.8152182 ])

In [135]:
word_vectors /= norm(word_vectors, axis=1).reshape(-1,1)

In [136]:
norm(word_vectors, axis=1)

array([1., 1., 1., ..., 1., 1., 1.])

##### 3.4.2.1 Baseline model

To evaluate embeddings we will stick to extrinsic evaluation - evaluation on real task - text classification

In [137]:
embeddings_lookup = dict(zip(ppmi_dataframe.index.values, word_vectors))

In [138]:
def sent2vec(tok_sentence, embeddings_lookup):
    word_embeddings = []
    for word in tok_sentence:
        try:
            word_embeddings.append(embeddings_lookup[word])
        except:
            continue
    word_embeddings = np.array(word_embeddings)
    v = word_embeddings.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(100)
    return v / np.sqrt((v ** 2).sum())

In [140]:
X_train_cooc_ppmi = [sent2vec(tokenized_sentence, embeddings_lookup) for tokenized_sentence in tqdm(tokenized_train)]
X_val_cooc_ppmi = [sent2vec(tokenized_sentence, embeddings_lookup) for tokenized_sentence in tqdm(tokenized_val)]

  0%|          | 0/17621 [00:00<?, ?it/s]

  0%|          | 0/1958 [00:00<?, ?it/s]

In [141]:
logreg_cooc_counts = LogisticRegression(solver='sag', max_iter=10000)
logreg_cooc_counts.fit(X_train_cooc_ppmi, train['author'])

LogisticRegression(max_iter=10000, solver='sag')

In [142]:
pred = logreg_cooc_counts.predict(X_val_cooc_ppmi)
f1 = f1_score(pred, val['author'], average='macro')
f1

0.7644805259796094

### TODO: 
* Finish item 3.1.4 about using stopwords 
* Add the reason to use logarithms for tf-idf (sublinear tf, etc.) [1](https://stats.stackexchange.com/questions/161640/understanding-the-use-of-logarithms-in-the-tf-idf-logarithm) and [2](https://stackoverflow.com/questions/27067992/why-is-log-used-when-calculating-term-frequency-weight-and-idf-inverse-document)
* Write down questions
    * bag of words
    * tf-idf
* Fill items 3.3-3.4 with information and refactor the code
    

In [61]:
np.log2(1_000_000)

19.931568569324174

In [62]:
np.log2(2_000_000)

20.931568569324174